In [ ]:
import cv2
import pytesseract
from pytesseract import Output, TesseractError
from bs4 import BeautifulSoup
import numpy as np
import os

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
os.environ['TESSDATA_PREFIX'] = r"C:\Program Files\Tesseract-OCR\tessdata"

cap = cv2.VideoCapture(r"C:\Users\gilbert\PQの最後まではやらない神々の悪戯ハデス√　＃２ [RbGcdI_DjqU].mp4")
soup = BeautifulSoup('<html><body></body></html>', 'html.parser')
body = soup.find('body')

prev_frame = None
frame_count = 0

# specific video's name and text box coordinates
x1 = 147
y1 = 427
x2 = 690
y2 = 472

x3 = 132
y3 = 511
x4 = 1078
y4 = 677

In [ ]:

# Define the region of interest (ROI) coordinates for the name and text boxes
roi_name_top_left = (x1, y1)  # coordinates of the top left corner of the name box
roi_name_bottom_right = (x2, y2)  # coordinates of the bottom right corner of the name box

roi_text_top_left = (x3, y3)  # coordinates of the top left corner of the text box
roi_text_bottom_right = (x4, y4)  # coordinates of the bottom right corner of the text box

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % 30 != 0:  # frameskip
        continue

    # Crop the frame to the ROIs
    frame_name = frame[roi_name_top_left[1]:roi_name_bottom_right[1], roi_name_top_left[0]:roi_name_bottom_right[0]]
    frame_text = frame[roi_text_top_left[1]:roi_text_bottom_right[1], roi_text_top_left[0]:roi_text_bottom_right[0]]

    # Process each ROI separately
    for roi_frame in [frame_name, frame_text]:
        # Compare current frame with the previous one
        if prev_frame is not None and np.array_equal(roi_frame, prev_frame):
            continue

        try:
            d = pytesseract.image_to_string(roi_frame, lang='jpn', config='--psm 6', output_type=Output.DICT)
            p = soup.new_tag('p')
            p.string = d['text']
            body.append(p)
            print(p)
        except TesseractError as e:
            print("An error occurred during OCR:", e)

        # Update the previous frame
        prev_frame = roi_frame

with open('output.html', 'w', encoding='utf-8') as f:
    f.write(str(soup))

cap.release()
cv2.destroyAllWindows()
